In [ ]:
from preamble import *

In [ ]:
from datetime import datetime, time

# 전기 요금 정보 (인상된 요금 반영)
electricity_rates = {
    'summer': {
        'off_peak': 64.37,  # 경부하 요금 (원/kWh)
        'mid_peak': 92.46,  # 중간부하 요금 (원/kWh)
        'on_peak': 123.88   # 최대부하 요금 (원/kWh)
    },
    'spring_fall': {
        'off_peak': 64.37,  # 경부하 요금 (원/kWh)
        'mid_peak': 69.50,  # 중간부하 요금 (원/kWh)
        'on_peak': 86.88    # 최대부하 요금 (원/kWh)
    },
    'winter': {
        'off_peak': 71.88,  # 경부하 요금 (원/kWh)
        'mid_peak': 90.80,  # 중간부하 요금 (원/kWh)
        'on_peak': 116.47   # 최대부하 요금 (원/kWh)
    }
}

# 시간대 구분
time_periods = {
    'summer': {
        'off_peak': [(time(23, 0), time(9, 0))],
        'mid_peak': [(time(9, 0), time(11, 0)), (time(12, 0), time(13, 0)), (time(17, 0), time(23, 0))],
        'on_peak': [(time(11, 0), time(12, 0)), (time(13, 0), time(17, 0))]
    },
    'spring_fall': {
        'off_peak': [(time(23, 0), time(9, 0))],
        'mid_peak': [(time(9, 0), time(11, 0)), (time(12, 0), time(13, 0)), (time(17, 0), time(23, 0))],
        'on_peak': [(time(11, 0), time(12, 0)), (time(13, 0), time(17, 0))]
    },
    'winter': {
        'off_peak': [(time(23, 0), time(9, 0))],
        'mid_peak': [(time(9, 0), time(10, 0)), (time(12, 0), time(17, 0)), (time(20, 0), time(22, 0))],
        'on_peak': [(time(10, 0), time(12, 0)), (time(17, 0), time(20, 0)), (time(22, 0), time(23, 0))]
    }
}

In [ ]:
import pickle

with open('best_polynomial_regression_model.pkl', 'rb') as f:
  model, poly = pickle.load(f)

In [ ]:
def calculate_power_from_flux(flux, model, poly):
    flux_poly = poly.fit_transform(np.array([[flux]]))
    power_kW = model.predict(flux_poly)[0]
    return power_kW

def get_season_and_period(dt):
    month = dt.month
    current_time = dt.time()
    
    if month in [7, 8]:
        season = 'summer'
    elif month in [11, 12, 1, 2]:
        season = 'winter'
    else:
        season = 'spring_fall'
    
    for period, times in time_periods[season].items():
        for start, end in times:
            if start <= current_time < end:
                return season, period
    return season, 'off_peak'

def calculate_daily_cost(data, model, poly):
    total_cost = 0
    for _, row in data.iterrows():
        season, period = get_season_and_period(row['datetime'])
        power_kW = calculate_power_from_flux(row['flux'], model, poly)
        cost_per_kWh = electricity_rates[season][period]
        energy_kWh = power_kW / 60
        total_cost += energy_kWh * cost_per_kWh
    return total_cost

def hourly_simulation(outflow, inflow_rate, v_min, v_max, current_storage):
    test_storage = current_storage
    for minute in range(60):
        test_storage += inflow_rate - outflow[minute]
        if test_storage < v_min:
            return 1
        elif test_storage > v_max:
            return -1
    return 0

In [ ]:
def optimize_multi_pump_flow(data_list, v_initial_list, capacity_list):
  num_reservoirs = len(data_list)
  minutes = len(data_list[0]['datetime'])
  
  pump_flows = [np.zeros(minutes) for _ in range(num_reservoirs)]
  over_flows = [np.zeros(minutes) for _ in range(num_reservoirs)]
  lower_flows = [np.zeros(minutes) for _ in range(num_reservoirs)]
  storage_levels = [v_initial for v_initial in v_initial_list]
  v_min_list = [capacity * 0.31 for capacity in capacity_list]
  v_max_list = [capacity * 0.94 for capacity in capacity_list]
  start_time = data_list[0]['datetime'][0]
    
    
  # 최적화 함수 정의
  def objective_function(hourly_flows):
    total_cost = 0
    for reservoir_idx in range(num_reservoirs):
      data = data_list[reservoir_idx]
      storage = v_initial_list[reservoir_idx]
      v_min = v_min_list[reservoir_idx]
      v_max = v_max_list[reservoir_idx]
      outflow = data['outflow'].values
      pump_flow = np.zeros(minutes)
      
      
      for minute in range(minutes):
        if minute % 60 == 0:
          hourly_flow = hourly_flows[reservoir_idx * 24 + minute // 60]

          t = minute // 60
          current_time = start_time + timedelta(hours=t)
          season, period = get_season_and_period(current_time)

          # 배수지 저장량 확인 및 조정(해당 분의 유입 유출 조정)
          if storage - np.sum(outflow[minute:minute+60] / 60 ) < v_min:
            hourly_flow = hourly_flow + (v_min - (storage - np.sum(outflow[minute:minute+60] / 60)))
          elif storage - np.sum(outflow[minute:minute+60] / 60) > v_max:
            hourly_flow = hourly_flow - ((storage - np.sum(outflow[minute:minute+60] / 60) ) - v_max)
          
          # 심야 시간 요금 절약을 위해 조정 (추가적인 충전 고려)
          if season == 'off_peak' and storage + hourly_flow - np.sum(outflow[minute:minute+60] / 60) <= v_max:
            hourly_flow += (v_max - (storage + hourly_flow - np.sum(outflow[minute:minute+60] / 60))) * 0.9  # 추가 충전
          elif season == 'mid_peak' and storage + hourly_flow - np.sum(outflow[minute:minute+60] / 60)  <= v_max:
            hourly_flow += (v_max - (storage + hourly_flow - np.sum(outflow[minute:minute+60] / 60))) * 0.6  # 추가 충전
          elif season == 'on_peak' and storage + hourly_flow - np.sum(outflow[minute:minute+60] / 60) >= v_min:
            hourly_flow -= ((storage + hourly_flow - np.sum(outflow[minute:minute+60] / 60)) - v_min) * 0.8 # 절감 충전
                    
        pump_flow[minute] = hourly_flow

        storage += pump_flow[minute] - outflow[minute]

      pump_flows[reservoir_idx] = pump_flow
        
        
    
    for reservoir_idx in range(num_reservoirs):
      data = data_list[reservoir_idx]
      storage = v_initial_list[reservoir_idx]
      v_min = v_min_list[reservoir_idx]
      v_max = v_max_list[reservoir_idx]
      outflow = data['outflow'].values
      pump_flow = pump_flows[reservoir_idx]
      over_flow = over_flows[reservoir_idx]
      lower_flow = lower_flows[reservoir_idx]
      
      for minute in range(minutes):
          storage += pump_flow[minute] - outflow[minute]
          if (storage > v_max):
              over_flow[minute] = storage - v_max
          elif (storage < v_min):
              lower_flow[minute] = v_min - storage

      over_non_zero_indices =  np.nonzero(over_flow)
      lower_non_zero_indices = np.nonzero(lower_flow)

      hourly_over_amount = np.zeros(24)
      hourly_lower_amount = np.zeros(24)
      
      if len(over_non_zero_indices[0]) != 0:
          for idx in over_non_zero_indices[0]:
              # print(f'{idx}분 over')
              h = idx // 60
              hourly_over_amount[h] -= over_flow[idx]
      if len(lower_non_zero_indices[0]) != 0:
          for idx in lower_non_zero_indices[0]:
              # print(f'{idx}분 lower')
              h = idx // 60
              hourly_lower_amount[h] += lower_flow[idx]

      adjustment = hourly_over_amount + hourly_lower_amount
      # print(adjustment)

      for i, value in enumerate(adjustment):
          if value == 0: continue
          else:
            pump_flows[reservoir_idx][i * 60 : (i + 1) * 60] += value / 60

    # 비용 계산
    for reservoir_idx in range(num_reservoirs):
      data = data_list[reservoir_idx]
      total_cost += calculate_daily_cost(data.assign(flux=pump_flows[reservoir_idx]), model, poly)
        
    return total_cost

    
  initial_guess = np.array([np.mean(data['outflow']) for data in data_list] * 24)
  
  # 각 배수지에 대한 유량 범위를 설정
  bounds = [(0, 10) for _ in range(num_reservoirs * 24)]

  result = minimize(objective_function, initial_guess, method='L-BFGS-B', bounds=bounds)
  optimized_flows = result.x
  
  
  for reservoir_idx in range(num_reservoirs):
    for minute in range(minutes):
      if minute % 60 == 0:
          hourly_flow = optimized_flows[reservoir_idx * 24 + minute // 60]
          pump_flows[reservoir_idx][minute] = hourly_flow

    
  return pump_flows

In [ ]:
# 예시 데이터 생성
num_reservoirs = 2
data_list = []
v_initial_list = []
capacity_list = []

for i in range(num_reservoirs):
  data_one_day = pd.DataFrame({
    'datetime': pd.to_datetime([datetime(2024, 7, 1, 0, i) for i in range(24 * 60)]),
    'outflow': np.random.rand(24 * 60) * 0.5 + 0.3,
    'pressure': np.random.rand(24 * 60) * 5 + 10,
    'flux': np.random.rand(24 * 60) * 0.3 + 0.1,
    'power': np.random.rand(24 * 60) * 5 + 10 
  })
  data_list.append(data_one_day)
  v_initial_list.append(1400 + i*100)
  capacity_list.append(2000 + i*100)

# 모델 학습
model, poly = train_power_model(pd.concat(data_list))

# 최적화된 유량 계산
optimized_flows = optimize_multi_pump_flow(data_list, v_initial_list, capacity_list)
for i, optimized_flow in enumerate(optimized_flows):
  print(f"배수지 {i+1} 최적 유량:", optimized_flow)

# 비용 계산 및 비교
total_cost_before = 0
total_cost_after = 0
for i, data in enumerate(data_list):
  cost_before = calculate_daily_cost(data, model, poly)
  cost_after = calculate_daily_cost(data.assign(flux=optimized_flows[i]), model, poly)
  total_cost_before += cost_before
  total_cost_after += cost_after

print(f"최적화 전 총 비용: {total_cost_before:,.0f} KRW")
print(f"최적화 후 총 비용: {total_cost_after:,.0f} KRW")
print(f"총 절감 비용: {total_cost_before - total_cost_after:,.0f} KRW")
print(f"절감률: {(1 - (total_cost_after / total_cost_before)) * 100:.2f}%")